# **AI TECH INSTITUTE** · *Intermediate AI & Data Science*
### Unsupervised Learning Lifecycle - Part 1
**Instructor:** Amir Charkhi | **Focus:** Data → Preprocessing → Dimensionality Reduction

---

## 🎯 The Complete Unsupervised Learning Workflow

```
┌─────────────────────────────────────────────────────────────────────┐
│                    UNSUPERVISED LEARNING LIFECYCLE                  │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│  PART 1 (This Notebook):                                           │
│  ┌──────────┐    ┌──────────────┐    ┌─────────────────────┐       │
│  │ Raw Data │ →  │ Preprocessing │ →  │ Dim Reduction (PCA) │       │
│  └──────────┘    └──────────────┘    └─────────────────────┘       │
│                                                                     │
│  PART 2 (Next Notebook):                                           │
│  ┌────────────┐    ┌────────────┐    ┌──────────────────┐          │
│  │ Clustering │ →  │ HP Tuning  │ →  │ Business Insights │          │
│  └────────────┘    └────────────┘    └──────────────────┘          │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

---

## 📚 What You'll Learn in Part 1

- Load and explore real-world data
- Build preprocessing pipeline
- Understand WHY dimensionality reduction matters
- Apply PCA for feature reduction
- Use t-SNE for visualization
- Prepare data for clustering

---

## 1. Setup

In [ ]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.pipeline import Pipeline

---
## 2. The Dataset: Wine Recognition

### 🍷 Business Context

**Scenario:** A wine distributor has chemical analysis data for wines from three different cultivars (grape varieties). They want to:
- Group wines by chemical profile (without using cultivar labels)
- Understand what makes wines similar/different
- Create marketing segments

**The Challenge:**
```
13 chemical features → Hard to visualize!
No labels provided → Unsupervised learning needed!
```

---

In [ ]:
from sklearn.datasets import load_wine

wine = load_wine()
X = pd.DataFrame(wine.data, columns=wine.feature_names)
y_true = wine.target  # We'll use this ONLY for evaluation, not training!

In [ ]:
X.shape

💡 **178 wines, 13 chemical features** - too many dimensions to visualize directly!

In [ ]:
X.head()

### 📊 Feature Overview

In [ ]:
X.describe().round(2)

💡 **Notice the different scales!**
- `proline`: 278 to 1680
- `magnesium`: 70 to 162
- `nonflavanoid_phenols`: 0.13 to 0.66

**This is why we need scaling!**

---

## 3. Exploratory Data Analysis

### 🔍 Check for Issues

In [ ]:
# Missing values?
X.isnull().sum().sum()

In [ ]:
# Data types
X.dtypes.value_counts()

✅ **Clean data:** No missing values, all numeric features

### 📈 Feature Distributions

In [ ]:
# Select 4 key features for visualization
key_features = ['alcohol', 'malic_acid', 'flavanoids', 'proline']

fig = make_subplots(rows=2, cols=2, subplot_titles=key_features)

for i, feat in enumerate(key_features):
    row, col = (i // 2) + 1, (i % 2) + 1
    fig.add_trace(
        go.Histogram(x=X[feat], nbinsx=20, name=feat, showlegend=False),
        row=row, col=col
    )

fig.update_layout(title='Feature Distributions', height=500, template='plotly_white')
fig.show()

### 🔗 Feature Correlations

In [ ]:
corr_matrix = X.corr()

fig = px.imshow(
    corr_matrix,
    text_auto='.2f',
    color_continuous_scale='RdBu_r',
    title='Feature Correlation Matrix',
    width=900,
    height=800
)
fig.show()

💡 **Key Insight:** Many features are correlated!
- `flavanoids` ↔ `total_phenols`: 0.86
- `od280/od315_of_diluted_wines` ↔ `flavanoids`: 0.79

**Correlated features = redundant information → PCA can help!**

---

## 4. Preprocessing Pipeline

### 🔧 Why Preprocessing Matters

```
Problem: Different scales

Feature A: [0.1, 0.2, 0.3]      (small values)
Feature B: [1000, 2000, 3000]   (large values)

Distance calculation:
  Feature B dominates! Feature A is ignored.

Solution: StandardScaler
  All features → mean=0, std=1
  Equal contribution to distance
```

---

In [ ]:
# Create scaler
scaler = StandardScaler()

In [ ]:
# Fit and transform
X_scaled = scaler.fit_transform(X)

In [ ]:
# Verify scaling worked
pd.DataFrame({
    'Feature': X.columns[:4],
    'Original Mean': X.iloc[:, :4].mean().round(2).values,
    'Scaled Mean': X_scaled[:, :4].mean(axis=0).round(6),
    'Original Std': X.iloc[:, :4].std().round(2).values,
    'Scaled Std': X_scaled[:, :4].std(axis=0).round(2)
})

✅ **All features now have mean≈0 and std≈1**

---

## 5. Why Dimensionality Reduction?

### ❓ The Problem: Curse of Dimensionality

```
With 13 features:

1. Can't visualize (we see in 2D/3D only)
2. Distance metrics become unreliable
3. Clustering algorithms struggle
4. Many features may be redundant (correlated)
```

### 💡 The Solution: Reduce Dimensions

```
Two main purposes:

1. VISUALIZATION (t-SNE)
   13D → 2D for plotting
   See if natural groups exist

2. FEATURE REDUCTION (PCA)
   13D → fewer D for modeling
   Remove redundancy
   Improve clustering
```

---

## 6. PCA: Principal Component Analysis

### 📖 What PCA Does

```
Finds new axes (principal components) that:
1. Capture maximum variance
2. Are uncorrelated with each other
3. Ordered by importance

Result:
  PC1: Most variance
  PC2: Second most (perpendicular to PC1)
  PC3: Third most (perpendicular to PC1 & PC2)
  ...
```

---

### 🔍 Step 1: How Many Components Do We Need?

In [ ]:
# Fit PCA with all components first
pca_full = PCA()
pca_full.fit(X_scaled)

In [ ]:
# Calculate cumulative variance
cumulative_var = np.cumsum(pca_full.explained_variance_ratio_)

# Find components for 95% variance
n_components_95 = np.argmax(cumulative_var >= 0.95) + 1
n_components_95

💡 **Only need ~10 components to capture 95% of information!**

In [ ]:
# Visualize variance explained
fig = go.Figure()

# Individual variance bars
fig.add_trace(go.Bar(
    x=list(range(1, 14)),
    y=pca_full.explained_variance_ratio_,
    name='Individual',
    marker_color='steelblue'
))

# Cumulative line
fig.add_trace(go.Scatter(
    x=list(range(1, 14)),
    y=cumulative_var,
    name='Cumulative',
    mode='lines+markers',
    line=dict(color='red', width=3),
    yaxis='y2'
))

# 95% threshold line
fig.add_hline(y=0.95, line_dash='dash', line_color='green', 
              annotation_text='95% threshold', yref='y2')

fig.update_layout(
    title='PCA: Variance Explained by Component',
    xaxis_title='Principal Component',
    yaxis_title='Variance Ratio',
    yaxis2=dict(title='Cumulative Variance', overlaying='y', side='right', range=[0, 1.05]),
    template='plotly_white',
    height=500,
    width=900
)
fig.show()

### 📊 Step 2: Apply PCA for Visualization (2D)

In [ ]:
# PCA to 2 dimensions for visualization
pca_2d = PCA(n_components=2)
X_pca_2d = pca_2d.fit_transform(X_scaled)

In [ ]:
# Variance captured by 2 components
var_2d = pca_2d.explained_variance_ratio_.sum()
f"{var_2d:.1%} variance captured in 2D"

In [ ]:
# Create DataFrame for plotting
df_pca = pd.DataFrame({
    'PC1': X_pca_2d[:, 0],
    'PC2': X_pca_2d[:, 1],
    'True_Label': y_true  # Only for visualization validation!
})

In [ ]:
# Visualize - WITHOUT labels (unsupervised view)
fig = px.scatter(
    df_pca, x='PC1', y='PC2',
    title=f'PCA: 13D → 2D ({var_2d:.1%} variance) - Can You See Groups?',
    template='plotly_white',
    width=800, height=600
)
fig.update_traces(marker=dict(size=10, color='steelblue', opacity=0.7))
fig.show()

💡 **Can you see natural groupings?** Let's verify with true labels (just for validation):

In [ ]:
# Visualize WITH true labels (validation only!)
fig = px.scatter(
    df_pca, x='PC1', y='PC2',
    color=df_pca['True_Label'].astype(str),
    title='PCA with True Labels (Validation Only - We Pretend Not to Know These!)',
    template='plotly_white',
    width=800, height=600,
    color_discrete_sequence=['#e41a1c', '#377eb8', '#4daf4a']
)
fig.update_traces(marker=dict(size=10, opacity=0.7))
fig.show()

✅ **PCA reveals the underlying structure!** Three groups are visible.

---

### 🔍 Step 3: Understand What PCA Learned

In [ ]:
# Feature loadings - what contributes to each PC?
loadings = pd.DataFrame(
    pca_2d.components_.T,
    columns=['PC1', 'PC2'],
    index=X.columns
).round(3)

loadings

In [ ]:
# Visualize loadings
fig = px.bar(
    loadings.reset_index().melt(id_vars='index'),
    x='index', y='value', color='variable',
    barmode='group',
    title='PCA Feature Loadings: What Drives Each Component?',
    labels={'index': 'Feature', 'value': 'Loading', 'variable': 'Component'},
    template='plotly_white',
    height=500, width=1000
)
fig.update_xaxes(tickangle=45)
fig.show()

💡 **Interpretation:**
- **PC1:** Driven by flavanoids, phenols, proline (quality indicators)
- **PC2:** Driven by color_intensity, alcalinity, malic_acid (acidity/color)

---

## 7. t-SNE: Better Visualization

### 📖 PCA vs t-SNE

```
PCA:                          t-SNE:
- Linear transformation       - Non-linear
- Preserves global structure  - Preserves LOCAL structure
- Fast                        - Slower
- Good for preprocessing      - Good for VISUALIZATION only
```

---

In [ ]:
# Apply t-SNE
tsne = TSNE(n_components=2, perplexity=30, random_state=42)
X_tsne = tsne.fit_transform(X_scaled)

In [ ]:
# Create DataFrame
df_tsne = pd.DataFrame({
    'Dim1': X_tsne[:, 0],
    'Dim2': X_tsne[:, 1],
    'True_Label': y_true
})

In [ ]:
# Compare PCA vs t-SNE
fig = make_subplots(rows=1, cols=2, subplot_titles=('PCA (Linear)', 't-SNE (Non-linear)'))

# PCA
for label in [0, 1, 2]:
    mask = df_pca['True_Label'] == label
    fig.add_trace(
        go.Scatter(x=df_pca[mask]['PC1'], y=df_pca[mask]['PC2'],
                   mode='markers', name=f'Class {label}',
                   marker=dict(size=8, opacity=0.7),
                   showlegend=True),
        row=1, col=1
    )

# t-SNE
for label in [0, 1, 2]:
    mask = df_tsne['True_Label'] == label
    fig.add_trace(
        go.Scatter(x=df_tsne[mask]['Dim1'], y=df_tsne[mask]['Dim2'],
                   mode='markers', name=f'Class {label}',
                   marker=dict(size=8, opacity=0.7),
                   showlegend=False),
        row=1, col=2
    )

fig.update_layout(
    title='Dimensionality Reduction Comparison',
    template='plotly_white',
    height=500, width=1100
)
fig.show()

💡 **t-SNE gives clearer cluster separation!** 

But remember: t-SNE is for **visualization only**, not for feature engineering.

---

## 8. Prepare Data for Clustering

### 🎯 Which Representation to Use?

```
Option 1: Original scaled features (13D)
  - All information preserved
  - May have noise and redundancy

Option 2: PCA-reduced features (e.g., 5D)
  - Removes redundancy
  - Keeps most variance
  - Often improves clustering

Option 3: t-SNE (2D)
  - NEVER use for clustering!
  - Only for visualization
```

**Best Practice:** Try both original and PCA-reduced, compare results!

---

In [ ]:
# Create PCA with optimal components for clustering
pca_cluster = PCA(n_components=0.90)  # Keep 90% variance
X_pca_cluster = pca_cluster.fit_transform(X_scaled)

In [ ]:
X_pca_cluster.shape

💡 **Reduced from 13 to 6 dimensions while keeping 90% of variance!**

In [ ]:
# Save prepared data for Part 2
prepared_data = {
    'X_scaled': X_scaled,           # Original scaled (13D)
    'X_pca': X_pca_cluster,         # PCA reduced (6D)
    'X_pca_2d': X_pca_2d,           # For visualization (2D)
    'X_tsne': X_tsne,               # For visualization (2D)
    'y_true': y_true,               # True labels (for evaluation only)
    'feature_names': X.columns.tolist()
}

# Save to file for Part 2
np.savez('prepared_wine_data.npz', **prepared_data)

---
## 9. Summary: Part 1 Complete!

### ✅ What We Did

```
┌─────────────────────────────────────────────────────────────┐
│  Raw Data (178 wines × 13 features)                        │
│       ↓                                                     │
│  EDA: Checked distributions, correlations                   │
│       ↓                                                     │
│  Preprocessing: StandardScaler (mean=0, std=1)              │
│       ↓                                                     │
│  Dimensionality Reduction:                                  │
│    • PCA (90% variance) → 6 dimensions for clustering       │
│    • PCA (2D) → visualization                               │
│    • t-SNE (2D) → visualization                             │
│       ↓                                                     │
│  Ready for Clustering! (Part 2)                             │
└─────────────────────────────────────────────────────────────┘
```

### 🔑 Key Takeaways

**1. Preprocessing is Critical**
- Scale features before distance-based methods
- Different scales = biased results

**2. Dimensionality Reduction Serves Two Purposes**
- Visualization (PCA or t-SNE to 2D)
- Feature reduction (PCA to fewer dimensions)

**3. PCA vs t-SNE**
- PCA: Fast, linear, good for preprocessing
- t-SNE: Non-linear, better separation, visualization ONLY

**4. Natural Groups Exist**
- Visualization showed 3 distinct clusters
- Now we need to find them automatically!

---

### 🚀 Next: Part 2

In Part 2, we will:
- Apply multiple clustering algorithms
- Tune hyperparameters systematically
- Evaluate and compare methods
- Select the best model
- Extract business insights

---

**AI Tech Institute** | *Building Tomorrow's AI Engineers Today*